# LangChain Quickstart

In this quickstart you will create a simple LangChain RAG and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/langchain_rag_quickstart.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
# pip install trulens-eval==0.18.0 langchain openai faiss-cpu tiktoken chromadb

### Add API keys
For this quickstart, you will need Open AI keys. The OpenAI key is used for embeddings, completion and evaluations.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."

### Import from LangChain

In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub

from trulens_eval import Feedback, Tru, TruChain
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

tru = Tru()

### Create Simple RAG with Langchain

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [ ]:
loader = WebBaseLoader("http://paulgraham.com/worked.html")

full_text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=524, chunk_overlap=100)
splits = text_splitter.split_documents(full_text)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:
retrieved_docs = retriever.invoke(
    "What did the author do growing up?"
)
print(retrieved_docs[0].page_content)

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke(input="what did the author do growing up")

### Send your first request

In [ ]:
rag_chain.invoke("What did the author do growing up?")

## Initialize Feedback Function(s)

In [ ]:
import numpy as np

# Initialize provider class
openai = OpenAI()

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

'''

grounded = Groundedness(groundedness_provider=OpenAI())

# Define a groundedness feedback function
f_groundedness = Feedback(grounded.groundedness_measure_with_cot_reasons).on(
    TruLlama.select_source_nodes().node.text
    ).on_output(
    ).aggregate(grounded.grounded_statements_aggregator)



# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

'''

## Instrument app for logging with TruLens

In [ ]:
tru_query_engine_recorder = TruChain(rag_chain,
    app_id='LangChain RAG v1',
    feedbacks=[f_qa_relevance])

In [ ]:
# or as context manager
with tru_query_engine_recorder as recording:
    query_engine.query("What did the author do growing up?")

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

Note: Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all